In [1]:
# import library
import pandas as pd
import numpy as np
import os

In [11]:
# import file
dataframe_prices = pd.read_csv('prices.csv')
dataframe_sales = pd.read_csv('sales.csv')
dataframe_prices['updated_at'][dataframe_prices['product_id'] == 3954203]

1    9/11/18 11:54
2    9/17/18 22:59
3     9/15/18 3:49
4    9/10/18 16:32
Name: updated_at, dtype: object

In [3]:
# convert object to date and sort 2 dataframe and check for null value
dataframe_prices['updated_at'] = pd.to_datetime(dataframe_prices['updated_at'])
dataframe_sales['ordered_at'] = pd.to_datetime(dataframe_sales['ordered_at'])
dataframe_prices

,product_id,old_price,new_price,updated_at
0,64,270000,239000,2018-09-10 16:37:00
1,3954203,60000,64000,2018-09-11 11:54:00
2,3954203,60500,57500,2018-09-17 22:59:00
3,3954203,64000,60500,2018-09-15 03:49:00
4,3954203,68800,60000,2018-09-10 16:32:00
5,3998909,15500,16500,2018-09-16 05:09:00
6,3998909,17000,15500,2018-09-13 06:43:00
7,3998909,19000,17000,2018-09-10 16:35:00
8,4085861,53500,52000,2018-09-17 22:59:00
9,4085861,53500,67000,2018-09-12 03:51:00


In [4]:
result = pd.merge_asof(dataframe_sales.sort_values(['ordered_at']), dataframe_prices.sort_values(['updated_at']), 
                                     left_on = 'ordered_at', right_on = 'updated_at', by = 'product_id', suffixes = ('_sales','_prices'), 
                                             direction = 'nearest')
result['revenue']=np.where(result['ordered_at']>result['updated_at'], result['new_price'], result['old_price'])
result
result

,product_id,ordered_at,quantity_ordered,old_price,new_price,updated_at,revenue
0,3954203,2018-09-11 01:43:00,1,68800,60000,2018-09-10 16:32:00,60000
1,4085861,2018-09-11 06:26:00,1,60000,53500,2018-09-11 08:51:00,60000
2,4085861,2018-09-11 06:53:00,1,60000,53500,2018-09-11 08:51:00,60000
3,4085861,2018-09-11 08:24:00,1,60000,53500,2018-09-11 08:51:00,60000
4,4085861,2018-09-11 09:30:00,1,60000,53500,2018-09-11 08:51:00,53500
...,...,...,...,...,...,...,...
170,4085861,2018-09-18 20:23:00,1,53500,52000,2018-09-17 22:59:00,52000
171,4085861,2018-09-18 20:43:00,1,53500,52000,2018-09-17 22:59:00,52000
172,4085861,2018-09-18 20:54:00,1,53500,52000,2018-09-17 22:59:00,52000
173,3954203,2018-09-18 21:26:00,1,60500,57500,2018-09-17 22:59:00,57500


In [5]:
# calculate total quantirty_ordered and total_revenue
final_df = result[['product_id','quantity_ordered','revenue']]
final_df = final_df.groupby(['product_id','revenue'], as_index=False)['quantity_ordered'].sum()
final_df['total_revenue'] = final_df['revenue']*final_df['quantity_ordered']
final_df

,product_id,revenue,quantity_ordered,total_revenue
0,64,239000,4,956000
1,3954203,57500,1,57500
2,3954203,60000,3,180000
3,3954203,64000,10,640000
4,3998909,15500,1,15500
5,3998909,16500,14,231000
6,3998909,17000,2,34000
7,4085861,52000,20,1040000
8,4085861,53500,40,2140000
9,4085861,58000,38,2204000
